In [ ]:
链接：https://www.jianshu.com/p/4905bf8e06e5

## SAVING AND LOADING MODELS

当提到保存和加载模型时，有三个核心功能需要熟悉：

1. torch.save：将序列化的对象保存到disk。这个函数使用Python的pickle实用程序进行序列化。使用这个函数可以保存各种对象的模型、张量和字典。
2. torch.load：使用pickle unpickle工具将pickle的对象文件反序列化为内存。
3. torch.nn.Module.load_state_dict:使用反序列化状态字典加载model’s参数字典。

## 一：WHAT IS A STATE_DICT

在PyTorch中，torch.nn.Module的可学习参数(即权重和偏差)，模块模型包含在model's参数中(通过model.parameters()访问)。state_dict是个简单的Python dictionary对象，它将每个层映射到它的参数张量。

**注意，只有具有可学习参数的层(卷积层、线性层等)才有model's state_dict中的条目。优化器对象(connector .optim)也有一个state_dict，其中包含关于优化器状态以及所使用的超参数的信息.**

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass,self).__init__()
        self.conv1=nn.Conv2d(3,6,5)
        self.pool=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(6,16,5)
        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
    def farward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=x.view(-1,16*5*5)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
# Initialize model
model=TheModelClass()
print(model)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [17]:
for name,param in model.named_parameters():
    print(name+'\t'+str(param.requires_grad))
    #print(type(param))
# Initialize optimizer
optimizer=torch.optim.SGD(model.parameters(),lr=1e-4,momentum=0.9)

print("\nModel's state_dict:")
# Print model's state_dict
for param_tensor in model.state_dict():
    print(param_tensor,"\t",model.state_dict()[param_tensor].size())
print("\noptimizer's state_dict:")
# Print optimizer's state_dict
for var_name in optimizer.state_dict():
    print(var_name,"\t",optimizer.state_dict()[var_name])

conv1.weight	True
conv1.bias	True
conv2.weight	True
conv2.bias	True
fc1.weight	True
fc1.bias	True
fc2.weight	True
fc2.bias	True
fc3.weight	True
fc3.bias	True

Model's state_dict:
conv1.weight 	 torch.Size([6, 3, 5, 5])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([16, 6, 5, 5])
conv2.bias 	 torch.Size([16])
fc1.weight 	 torch.Size([120, 400])
fc1.bias 	 torch.Size([120])
fc2.weight 	 torch.Size([84, 120])
fc2.bias 	 torch.Size([84])
fc3.weight 	 torch.Size([10, 84])
fc3.bias 	 torch.Size([10])

optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.0001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [1988292414920, 1988292413624, 1988292414776, 1988292412904, 1988292370792, 1988292371944, 1988292370576, 1988292370936, 1988292373888, 1988292372232]}]


In [ ]:
torch.nn.no

## 二：SAVING & LOADING MODEL FOR INFERENCE

### Save/Load state_dict (Recommended)--只保存参数

- Save:
```
  torch.save(model.state_dict(), PATH)
```


在保存模型进行推理时，只需要保存训练过的模型的学习参数即可。一个常见的PyTorch约定是使用.pt或.pth文件扩展名保存模型。


- Load:
```
 model = TheModelClass(*args, **kwargs)
 model.load_state_dict(torch.load(PATH))
 model.eval()
```


记住，您必须调用model.eval()，以便在运行推断之前将dropout和batch规范化层设置为评估模式。如果不这样做，将会产生不一致的推断结果。

**Note:
 注意，load_state_dict()函数接受一个dictionary对象，而不是保存对象的路径。这意味着您必须在将保存的state_dict传至load_state_dict()函数之前反序列化torch.load()它。**

### Save/Load Entire Model

- Save:
```
  torch.save(model, PATH)
```


- Load:
```
#Model class must be defined somewhere
  model = torch.load(PATH)
 model.eval()
 ```


## 三：Example1：

### Save:

In [ ]:
torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        ...
        }, PATH)

### Load:

In [ ]:
model = TheModelClass(*args, **kwargs)
optimizer = TheOptimizerClass(*args, **kwargs)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - or -
model.train()</pre>

在保存用于推理或恢复训练的通用检查点时，必须保存模型的state_dict。另外，保存优化器的state_dict也是很重要的，因为它包含缓冲区和参数，这些缓冲区和参数是在模型训练时更新的。要保存多个组件，请将它们组织在字典中，并使用torch.save()序列化字典。一个常见的PyTorch约定是使用.tar文件扩展名保存这些检查点。

## Example2:初始化参数

### class torch.nn.Parameter()
一种Variable，被视为一个模块参数。

- Parameters 是 Variable 的子类。当与Module一起使用时，它们具有非常特殊的属性，当它们被分配为模块属性时，它们被自动添加到其参数列表中，并将出现在例如parameters()迭代器中。分配变量没有这样的效果。这是因为人们可能希望在模型中缓存一些临时状态，如RNN的最后一个隐藏状态。如果没有这样的班级Parameter，这些临时人员也会注册。

- 另一个区别是，parameters不能是volatile，他们默认要求梯度。

- 参数说明:

    - data (Tensor) – parameter tensor.

    - requires_grad (bool, optional) – 默认True

In [ ]:
def init_weights(self, pretrained='',):
    logger.info('=> init weights from normal distribution')
    for m in self.modules():
        #Conv2d层参数初始化:正态分布weight
        if isinstance(m, nn.Conv2d):
            nn.init.normal_(m.weight, std=0.001)
        #BatchNorm层参数初始化:常量weight=1,bias=0
        elif isinstance(m, InPlaceABNSync):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)
    #加载预训练模型的参数
    if os.path.isfile(pretrained):
        #先反序列化预训练模型保存的预训练参数字典
        pretrained_dict = torch.load(pretrained)
        logger.info('=> loading pretrained model {}'.format(pretrained))
        #当前模型所有参数的字典
        model_dict = self.state_dict()
        #在预训练参数字典中保留当前模型存在的参数
        pretrained_dict = {k: v for k, v in pretrained_dict.items()
                           if k in model_dict.keys()}
        for k, _ in pretrained_dict.items():
            logger.info(
                '=> loading {} pretrained model {}'.format(k, pretrained))
        #更新当前模型参数字典
        model_dict.update(pretrained_dict)
        #加载
        self.load_state_dict(model_dict)